In [ ]:
!conda install torch fisher 

In [ ]:
import numpy as np
import torch
from fisher.gmm import GMM
from fisher.fv import FisherVector
import cv2
import numpy as np

Theres 917 actions in this demonstration  
So each feature is a 512 long vector for each frame

Each video has a feature (for each frame), and a groundtruth file that shows action at each step.   
Mapping is a file that has a number - action mapping


In [ ]:


def extract_sift_features(image_path):
    """Extract SIFT descriptors from an image."""
    # Load the image
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if image is None:
        raise FileNotFoundError(f"Image not found: {image_path}")
    
    # Initialize SIFT
    sift = cv2.SIFT_create()
    
    # Detect and compute SIFT keypoints and descriptors
    _, descriptors = sift.detectAndCompute(image, None)
    
    return descriptors

def train_gmm(features, num_clusters=64, max_iters=100):
    """Train a GMM using the fisher package."""
    # Convert to PyTorch tensor
    features = torch.tensor(features, dtype=torch.float32)
    
    # Train GMM
    gmm = GMM(k=num_clusters, n_iter=max_iters)
    gmm.fit(features)
    return gmm

def compute_fisher_vector(features, gmm):
    """Compute the Fisher vector of the features given a GMM."""
    # Convert features to PyTorch tensor
    features = torch.tensor(features, dtype=torch.float32)
    
    # Initialize Fisher Vector encoder
    fv = FisherVector(gmm)
    
    # Compute Fisher vector
    fisher_vector = fv(features)
    return fisher_vector.cpu().numpy()

# Example usage
if __name__ == "__main__":
    # Path to the image
    image_path = "example.jpg"
    
    # Step 1: Extract SIFT features
    descriptors = extract_sift_features(image_path)
    if descriptors is None or len(descriptors) == 0:
        raise ValueError("No descriptors found in the image.")
    
    # Step 2: Train GMM (using descriptors from one or multiple images)
    gmm = train_gmm(descriptors, num_clusters=64)
    
    # Step 3: Compute Fisher vector
    fv = compute_fisher_vector(descriptors, gmm)
    
    print("Fisher Vector:", fv)
    print("Fisher Vector Dimension:", fv.shape)
